In [ ]:
import pandas as pd #Python Data Analysis Library
import numpy as np #Python Scientific Library (Umumnya membantu
#dalam urusan list)

#Selective import modul-modul Scikit Learn
#(Scikit Learn memiliki banyak modul machine learning)
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!wget https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip

In [ ]:
import os                                                                         #Mengimpor modul os mengakses folder data
import zipfile                                                                    #Mengimpor modul zipfile untuk membaca file zip

local_zip = 'rockpaperscissors.zip'                                      #Mendefinisikan file zip yang akan diekstrak
zip_ref = zipfile.ZipFile(local_zip, 'r')                                         #Mengekstrak file zip
zip_ref.extractall('rockpaperscissors')                                                   #Menempatkan file hasil ekstrak zip
zip_ref.close()

In [ ]:
import os                                                                         #Mengimpor modul os mengakses folder data
folder_scissors = os.path.join('rockpaperscissors/rockpaperscissors/scissors')                #Mendefinisikan Folder Training untuk gambar Gunting
folder_rock= os.path.join('rockpaperscissors/rockpaperscissors/rock')                        #Mendefinisikan Folder Training untuk gambar Batu
folder_paper = os.path.join('rockpaperscissors/rockpaperscissors/paper')                    #Mendefinisikan Folder Training untuk gambar Kertas

print('Total gambar scissors:', len(os.listdir(folder_scissors)))              #Menampilkan jumlah gambar di dalam folder Gunting
print('Total gambar rock:', len(os.listdir(folder_rock)))                    #Menampilkan jumlah gambar di dalam folder Batu
print('Total gambar kertas:', len(os.listdir(folder_paper)))                #Menampilkan jumlah gambar di dalam folder Kertas

print("Nama file gambar:")
file_scissor = os.listdir(folder_scissors)                                           #Mendaftar file di dalam folder gunting
print("Scissor:",file_scissor[:5])                                                #Menampilkan nama 5 file pertama di dalam folder gunting

file_rock = os.listdir(folder_rock)                                                 #Mendaftar file di dalam folder batu
print("Rock:",file_rock[:5])                                                      #Menampilkan nama 5 file pertama di dalam folder batu

file_paper = os.listdir(folder_paper)                                             #Mendaftar file di dalam folder kertas
print("Paper:",file_paper[:5])

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt                                                     #Mengimpor modul matplotlib
import matplotlib.image as mpimg

nrows = 3                                                                           #Jumlah baris tampilan gambar
ncols = 4                                                                           #Jumlah baris tampilan gambar

pic_index = 0                                                                        #Index awal untuk iterasi gambar

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 3)                                           #Mengatur gambar output matplotlib, dan ukurannya agar menjadi 4*6 gambar

pic_index += 4                                                                      #Index tambah bertambah 4 setiap baris
gambar_gunting  = [os.path.join(folder_scissors, fname)                              #Menampilkan gambar gunting
                      for fname in file_scissor[pic_index-4:pic_index]]
gambar_batu     = [os.path.join(folder_rock, fname)                               #Menampilkan gambar batu
                      for fname in file_rock[pic_index-4:pic_index]]
gambar_kertas   = [os.path.join(folder_paper, fname)                             #Menampilkan gambar kertas
                      for fname in file_paper[pic_index-4:pic_index]]

for i, img_path in enumerate(gambar_gunting+gambar_batu+gambar_kertas):
  sp = plt.subplot(nrows, ncols, i + 1)                                             #Mengatur subplot; indeks subplot dimulai dari 1
  sp.axis('Off')                                                                    #Tidak menampilkan Gridlines

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
import tensorflow as tf                                                           #Mengimpor modul tensorflow
import keras.preprocessing                                                        #Mengimpor modul keras
from keras.preprocessing import image                                             #Mengimpor keras_preprocessing untuk gambar
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_dir = "rockpaperscissors/novan/rps-cv-images/"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
#AUGMENTASI
	    rotation_range=20,                                                                         #Menerapkan skenario rotasi gambar hingga 40 derajat
      width_shift_range=0.3,                                                                     #Menerapkan skenario pelebaran gambar hingga hingga sebanyak 0.2 dari ukuran asli
      height_shift_range=0.3,                                                                    #Menerapkan skenario peninggian gambar hingga hingg sebanyak 0.2 dari ukuran asli
      shear_range=0.4,                                                                           #Menerapkan skenario pemangkasan gambar gambar hingga sebanyak 0.2 dari ukuran asli
      zoom_range=0.3,                                                                            #Menerapkan skenario zoom gambar hingga sebesar 0.2
      horizontal_flip=True,                                                                      #Menerapkan skenario putar balik gambar secara horizontal
      fill_mode='nearest',
#MEMECAH DATA MENJADI DATA TRAINING DAN DATA VALIDASI
      validation_split=0.4
      )

In [ ]:
train_generator = training_datagen.flow_from_directory(
	img_dir,
	target_size=(150,150),
  batch_size=32,
	class_mode='categorical',
  shuffle=True,
  subset='training'
  )

In [ ]:
validation_generator = training_datagen.flow_from_directory(                                 #Generator data validasi
	img_dir,                                                                                 #Folder target untuk digenerasi
	target_size=(150,150),
  batch_size=32,                                                                                          #Mengubah resolusi seluruh gambar menjadi 150x150 piksel
	class_mode='categorical',                                                                  #Kelas data kategori untuk klasifikasi multivariabel
  shuffle=True,
  subset='validation'                                                                        #Untuk data validasi
  )

In [ ]:
model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(128, activation='relu'),

    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()
Model: "sequential_2"

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Latih model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=4,  # Sesuaikan jumlah epoch sesuai kebutuhan
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)



In [ ]:
# Evaluasi model
accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy[1] * 100:.2f}%")

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():

  path = fn
  img = image.load_img(path, target_size=(150, 150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  if classes[0][0]==1:
    print('KERTAS')
  elif classes[0][1]==1:
    print('BATU')
  elif classes[0][2]==1:
    print('GUNTING')
  else:
    print('Ga tau')